In [ ]:
import numpy as np
from pathlib import Path

from pymor.basic import *
from pymor.core.pickle import load

from RBInvParam.problems.problems import build_InstationaryModelIP

set_log_levels({
    'pymor' : 'WARN'
})

set_defaults({})


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "cm",
    "font.size": 10,
    'text.latex.preamble': r'\usepackage{amsfonts} \usepackage{accents} \usepackage{mathrsfs}',
    'figure.dpi': 400
})

In [ ]:
#WORK_DIR = Path('/run/media/benedikt/38E84473425D444F/Dokumente/Studium/Mathematik/PhD/parabolische_inverse_probleme')
WORK_DIR = Path('/home/benedikt/Dokumente/parabolische_inverse_probleme')
data_dir_path = WORK_DIR / Path('experiments/paper_POD_tol_compare')

FOM_data_path = data_dir_path / 'reaction_FOM_300_non_time_dep'
exp_name_ = 'reaction_TR_non_time_dep_'
#TR_data_path = data_dir_path / 'diffusion_TR_non_time_dep_1e-9'

setup_path = FOM_data_path / 'setup.pkl'
optimizer_parameter_path = FOM_data_path / 'optimizer_parameter.pkl'


with open(setup_path, 'rb') as file:
    setup = load(file)

with open(optimizer_parameter_path, 'rb') as file:
    optimizer_parameter = load(file)

if not 'FOM' in locals():
    FOM, grid_data, assembled_products = build_InstationaryModelIP(setup=setup)

if FOM_data_path:
    with open(FOM_data_path / 'FOM_IRGNM_final.pkl', 'rb') as file:
        FOM_data = load(file)



In [ ]:
from RBInvParam.products import BochnerProductOperator

# q_FOM = data['FOM']['q'][-1]
# q_TR = data['TR']['q'][-1]

#nt = setup['dims']['nt']
nt = 1

def L2relerror(q_FOM, q_TR) -> float:
    bochner_L2 = BochnerProductOperator(
        product=NumpyMatrixOperator(
            matrix = assembled_products['l2'],
            source_id = FOM.Q.id,
            range_id = FOM.Q.id
        ),
        delta_t=setup['model_parameter']['delta_t'],
        space = FOM.Q,
        nt = nt
    )
    
    
    return np.sqrt(bochner_L2.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_L2.apply2(q_FOM, q_FOM)[0,0])


def H1relerror(q_FOM, q_TR) -> float:
    bochner_H1 = BochnerProductOperator(
        product=NumpyMatrixOperator(
            matrix = assembled_products['h1'],
            source_id = FOM.Q.id,
            range_id = FOM.Q.id
        ),
        delta_t=setup['model_parameter']['delta_t'],
        space = FOM.Q,
        nt = nt
    )
    
    
    return np.sqrt(bochner_H1.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_H1.apply2(q_FOM, q_FOM)[0,0])




In [ ]:
import pandas as pd

columns = ['Algorithm', 
           'eps POD',
           'L2relerror',
           'H1relerror',
           'time [s]', 
           'speed up', 
           'FOM solves', 
           r'$n_Q$', 
           r'$n_V$', 
           'o. iter']

rows = []
algo = 'FOM'
FOM_solves = FOM_data['FOM_num_calls']['solve_state'] + FOM_data['FOM_num_calls']['solve_adjoint'] + \
             FOM_data['FOM_num_calls']['solve_linearized_state'] + FOM_data['FOM_num_calls']['solve_linearized_adjoint']
row = [algo,
       '--',
       '--',
       '--',
       int(FOM_data['total_runtime'][-1]),
       '--',
       FOM_solves,
       '--',
       '--',
       len(FOM_data['J'])-1]
rows.append(row)

#exp_name_ = 'reaction_TR_non_time_dep_'
exp_names = [exp_name_ + id_ for id_ in ['1e-9', '1e-10', '1e-11', '1e-12', '1e-13', '1e-14']]

for exp_name in exp_names:
    with open(data_dir_path / exp_name / 'TR_IRGNM_final.pkl', 'rb') as file:
        TR_data = load(file)

    with open(data_dir_path / exp_name / 'optimizer_parameter.pkl', 'rb') as file:
        optimizer_parameter = load(file)

    assert optimizer_parameter['enrichment']['parameter_HaPOD_tol'] == optimizer_parameter['enrichment']['state_HaPOD_tol']

    algo = 'TR'
    FOM_solves = TR_data['FOM_num_calls']['solve_state'] + TR_data['FOM_num_calls']['solve_adjoint'] + \
                 TR_data['FOM_num_calls']['solve_linearized_state'] + TR_data['FOM_num_calls']['solve_linearized_adjoint']
    
    HaPOD_tol = optimizer_parameter['enrichment']['parameter_HaPOD_tol']
    L2relerror_ = L2relerror(FOM_data['q'][-1], TR_data['q'][-1])
    H1relerror_ = H1relerror(FOM_data['q'][-1], TR_data['q'][-1])
    row = [algo,
           f'{HaPOD_tol:.0e}',
           f'{L2relerror_:.2e}',
           f'{H1relerror_:.2e}',
           int(TR_data['total_runtime'][-1]),
           FOM_data['total_runtime'][-1] / TR_data['total_runtime'][-1],
           int(FOM_solves),
           TR_data['dim_Q_r'][-1],
           TR_data['dim_V_r'][-1],
           len(TR_data['J'])]
    rows.append(row)
    

df = pd.DataFrame.from_records(rows, columns=columns)

In [ ]:
df

In [ ]:
print(df.to_latex(index=False, float_format="%.2f", column_format='lc|cccccccc'))